In [1]:
import numpy as np
import cv2
import glob
import math

In [2]:
def ShowResizedIm(img,windowname,scale):
    """
    opencv imshow resized image on a new window
    Parameters:
        -img: image
        -window: window name
        -scale: size of the display image will be divided by this value(ex. scale=2 will make image 2 time smaller)
    """
    cv2.namedWindow(windowname, cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
    height, width = img.shape[:2]   #get image dimension
    cv2.resizeWindow(windowname,int(width/scale) ,int(height/scale))                    # Resize image
    cv2.imshow(windowname, img)                            # Show image

In [3]:
def findLineCoordinate(cY, cX, canv_H, canv_W,theta):
    """
    Find coordinates of both ends of a line from theta and a single point

    Parameters:
        -cY: an int of vertical coordinate of the single point
        -cX: an int of horizontal coordinate of the single point
        -canv_H: an int of hight of a canvas
        -canv_W: an int of width of a canvas
        -theta: a size of angle in degree measured form right-handed horizontal line of the point 

    Returns:
        -v1,u1,v2,u2: coordinate of both end of the line(v is vertical, u is horizontal)
    """
    if theta < 90:
        #find first coordinate of the line
        u1 = int(cY-(math.tan(math.radians(theta))*(canv_W-cX)))
        #clip of some part of the line that extend out of the image
        if u1 < 0:
            v1 = int(canv_W-1+(u1/math.tan(math.radians(theta))))
            u1 = 0
        else:
            v1 = canv_W-1
        #find second coordinate of the line
        u2 = int(cY+(math.tan(math.radians(theta))*cX))
        #clip of some part of the line that extend out of the image
        if u2 > canv_H-1:
            v2 = int((u2-canv_H-1)/math.tan(math.radians(theta)))
            u2 = canv_H-1
        else:
            v2 = 0
    elif theta == 90:
        #find first coordinate of the line
        u1 = 0
        v1 = cX
        #find second coordinate of the line
        u2 = canv_H
        v2 = cX
    else:
        #find first coordinate of the line
        u1 = int(cY+(math.tan(math.radians(theta))*(canv_W-1-cX)*(-1)))
        #clip of some part of the line that extend out of the image
        if u1 < 0:
            v1 = int(u1/math.tan(math.radians(theta)))
            u1 = 0
        else:
            v1 = canv_W-1
        #find second coordinate of the line
        u2 = int(cY-(math.tan(math.radians(theta))*cX*(-1)))
        #clip of some part of the line that extend out of the image
        if u2 > canv_H-1:
            v2 = int(canv_W-1+(u2-canv_H-1)/math.tan(math.radians(theta)))
            u2 = canv_H-1
        else:
            v2 = 0
    return v1,u1,v2,u2

In [4]:
def createLineIterator(P1, P2, img):
    """
    Produces and array that consists of the coordinates and intensities of each pixel in a line between two points

    Parameters:
        -P1: a numpy array that consists of the coordinate of the first point (x,y)
        -P2: a numpy array that consists of the coordinate of the second point (x,y)
        -img: the image being processed

    Returns:
        -it: a numpy array that consists of the coordinates and intensities of each pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
    """
    #define local variables for readability
    imageH = img.shape[0]
    imageW = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]

    #difference and absolute difference between points
    #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(int(dYa),int(dXa)),3),dtype=np.float32)
    itbuffer.fill(np.nan)

    #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

    #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<imageW) & (colY<imageH)]

    #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]

    return itbuffer

In [118]:
#http://www.gregbugaj.com/?p=524
class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def euclidean_distance(self, other):
        p1 = (other.x - self.x) ** 2
        p2 = (other.y - self.y) ** 2
        vd = (p1 + p2) ** 0.5
        return vd


class PointList(list):
    fraction = .7

    def hausdorffPHD(self, other):
        max_distance = 0
        ranking = []
        for i, a in enumerate(self):
            min_distance = float('inf')

            for j, b in enumerate(other):
                ed = a.euclidean_distance(b)

                if ed < min_distance:
                    min_distance = ed

            ranking.append(min_distance)
        ranking = sorted(ranking)
        k = int(len(self) * self.fraction)-1
        return ranking[k]


def hausdorff(seta, setb):
    habPHD = seta.hausdorffPHD(setb)
    hbaPHD = setb.hausdorffPHD(seta)
    distancePHD = max(habPHD, hbaPHD)
    #print(f'hd = {distancePHD:.4f}\t {habPHD:.4f}\t {hbaPHD:.4f}')
    return distancePHD


def main():
    seta = PointList()
    setb = PointList()

    seta.append(Point(100,1000))
    seta.append(Point(10,500))
    seta.append(Point(500,1000))
    setb.append(Point(-400,-1000))

    val = hausdorff(seta, setb)
    print(val)

if __name__ == '__main__':
    main()

2061.5528128088304


In [158]:
#find Vessel symmetry line degree
def findVSL(edges,showVSL = 1):
    degree_step = 5
    canv_H,canv_W = edges.shape[:2]
    #create mask to preview spining line
    mask = np.zeros((canv_H,canv_W), dtype=np.uint8)#test
    #Store VSL value for each line
    VSL = {}
    for theta in range(0,180,degree_step):
        #find line coordinate from theta
        mask_line_v1,mask_line_u1,mask_line_v2,mask_line_u2 = findLineCoordinate(cY, cX, canv_H, canv_W,theta)
        cv2.line(mask,(mask_line_v1,mask_line_u1),(mask_line_v2,mask_line_u2),(255,255,255),1)#test
        #find all pixels on the line
        line = createLineIterator(np.array((mask_line_v1,mask_line_u1)),np.array((mask_line_v2,mask_line_u2)), mask)
        #create mask to preview subline on the line
        submask = np.zeros((canv_H,canv_W), dtype=np.uint8)
        #find new theta for orthogonal subline which is in range 0-179
        if theta < 90:
            theta2 = theta+90
        else:  
            theta2 = theta-90
        #Store PHD output for each subline
        PHD_val = []
        mask_test = np.zeros((canv_H,canv_W), dtype=np.uint8)#test
        cv2.line(mask_test,(mask_line_v1,mask_line_u1),(mask_line_v2,mask_line_u2),(255,255,255),1)#test
        for pixel in line:
            #pixel[0] is x, pixel[1] is y
            sub_line_v1,sub_line_u1,sub_line_v2,sub_line_u2 = findLineCoordinate(pixel[1], pixel[0], canv_H, canv_W,theta2)
            cv2.line(submask,(sub_line_v1,sub_line_u1),(sub_line_v2,sub_line_u2),(255,255,255),1)      
            #crate set1 pm
            set1 = PointList()
            #crate list of pixels on a line contaied coordinate and value
            subline = createLineIterator(np.array((sub_line_v1,sub_line_u1)),np.array((pixel[0],pixel[1])), edges)
            #run through all pixel on subline
            if np.any(subline[:,2]):
                for subpixel in subline:
                    if subpixel[2] == 255:
                        #add vector of subpixel to set1
                        set1.append(Point(abs(pixel[0] - subpixel[0]),abs(pixel[1] - subpixel[1])))
                        mask_test[int(subpixel[1])][int(subpixel[0])] = 255#test
            #crate set2 pm
            set2 = PointList()
            #crate list of pixels on a line contaied coordinate and value
            subline = createLineIterator(np.array((sub_line_v2,sub_line_u2)),np.array((pixel[0],pixel[1])), edges)
            #run through all pixel on subline
            if np.any(subline[:,2]):
                for subpixel in subline:
                    if subpixel[2] == 255:
                        #add vector of subpixel to set2
                        set2.append(Point(abs(pixel[0] - subpixel[0]),abs(pixel[1] - subpixel[1])))
                        mask_test[int(subpixel[1])][int(subpixel[0])] = 255#test
            #find PHD
            if len(set1) and len(set2):
                distance = hausdorff(set1, set2)
                PHD_val.append(distance)

                if theta == 1000:
                    print(distance)
                    ShowResizedIm(np.hstack([mask_test,]),"mark",2)#test
                    cv2.waitKey(0)#test
                    cv2.destroyAllWindows()#test

        #find VSL
        if len(PHD_val):
            VSL[theta]= sum(PHD_val)/len(line)
        else:
            VSL[theta] = 0
        if showVSL:
            print(VSL[theta])
    #find the symetry line
    return min(VSL, key=VSL.get)


In [6]:
#=========USER START================
#folder path
path = 'bv image/*.jpg' 


#=========USER END================

In [63]:
image_list = []
for filename in glob.glob(path):
    image_list.append(filename)

name_list = []

In [125]:
img = cv2.imread(image_list[2])
ShowResizedIm(np.hstack([img,]),"mark",2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [126]:
edges = cv2.Canny(img,100,200)
ShowResizedIm(np.hstack([edges,]),"mark",2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [79]:
cv2.imwrite("temp image/fungi edges.jpg",edges)

True

In [127]:
canvas = edges.copy()
# calculate moments of binary image
M = cv2.moments(edges)
 
# calculate x,y coordinate of center
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])
 
# put text and highlight the center
cv2.circle(canvas , (cX, cY), 5, (255, 255, 255), -1)
cv2.putText(canvas , "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
ShowResizedIm(np.hstack([canvas,]),"mark",2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [159]:
VSL = findVSL(edges,showVSL = 0)

72.19034428175702
90.88087886308904
111.11031677443462
108.61239693803999
105.81813324531146
106.51041510491835
101.59293490060834
89.24192047208385
95.26700776798472
105.4286354330406
121.03029521157568
132.10502798982498
135.90882180638314
136.38851260174658
142.02823205302613
143.72693936904867
132.77752485533742
147.59161310750395
146.1891472868217
141.9288892895202
133.13455115136838
121.27713592178262
128.74627879577199
114.96747653753943
97.6168986629709
100.96035121534923
88.03893259628349
72.45313450659535
57.41826091312491
65.363368179656
70.97130794342053
68.34838129725979
72.8953056317973
71.02364163131193
75.58786189346115
68.7002395476111


In [138]:
sum1 = cv2.bitwise_or(canvas,canvas,mask = mask)
ShowResizedIm(np.hstack([canvas+mask,]),"mark",2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [160]:
VSL


140

140

In [38]:
len(PHD_val)

1103

In [40]:
cv2.imwrite("temp image/canvas.jpg",canvas)

True

In [12]:
cv2.imwrite("temp image/line mask.jpg",mask)

True